In [2]:
import numpy as np


In [3]:
import os, sys
sys.path.append('lib')

In [4]:
from RecommenderModel import RecommenderModel
from ModelConfig import ModelConfig
from tensorflow.keras.losses import BinaryCrossentropy,SparseCategoricalCrossentropy, MeanSquaredError
from tensorflow.keras.optimizers import Adam
from FileReader import FileReader
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

2024-05-19 16:36:09.893419: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-19 16:36:14.362508: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-05-19 16:36:14.364056: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [5]:
fr=FileReader('aiml_harveen')

In [22]:
item_train=fr.readText('movie/recommender/orig_data/content_item_train.csv')
user_train=fr.readText('movie/recommender/orig_data/content_user_train.csv')
y_train=fr.readText('movie/recommender/orig_data/content_y_train.csv')
item_vecs = fr.readText('movie/recommender/orig_data/content_item_vecs.csv', delimiter=',')


In [7]:
num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items


print(f"Number of training vectors: {len(item_train)}")

Number of training vectors: 50884


In [8]:
item_train_unscaled = item_train
user_train_unscaled = item_train
y_train_unscaled    = item_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))

In [9]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test data shape: {item_test.shape}")

movie/item training data shape: (40707, 17)
movie/item test data shape: (10177, 17)


In [10]:
modelConfig=ModelConfig(
    model_type="recommender",
    layers=[["relu","relu","linear"],["relu","relu","linear"]],
    regularizers=[[0.00,0.00,0.00],[0.00,0.00,0.00]],
    neurons=[[256,128,32],[256,128,32]],
    epochs=30,
    loss=MeanSquaredError(),
    optimizer=Adam(learning_rate=0.01),
    verbose=1,
    no_of_networks=2
)

2024-05-19 16:36:21.668128: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-05-19 16:36:21.669478: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-05-19 16:36:21.669577: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (instance-20240502-111332): /proc/driver/nvidia/version does not exist
2024-05-19 16:36:21.682770: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow 

In [11]:
recoModel = RecommenderModel(modelConfig)

In [12]:
recoModel.train([item_train,user_train],y_train)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 17)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 17)]         0           []                               
                                                                                                  
 sequential (Sequential)        (None, 32)           41632       ['input_1[0][0]']                
                                                                                                  
 sequential_1 (Sequential)      (None, 32)           41632       ['input_2[0][0]']                
                                                                                              

In [15]:
recoModel.model.evaluate([item_test,user_test],y_test)

319/319 [==============================] - 1s 2ms/step - loss: 0.0808


0.08080785721540451

In [16]:
new_user_id = 5000
new_rating_ave = 0.0
new_action = 0.0
new_adventure = 5.0
new_animation = 0.0
new_childrens = 0.0
new_comedy = 0.0
new_crime = 0.0
new_documentary = 0.0
new_drama = 0.0
new_fantasy = 5.0
new_horror = 0.0
new_mystery = 0.0
new_romance = 0.0
new_scifi = 0.0
new_thriller = 0.0
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])

In [35]:
user_vecs = np.tile(user_vec, (len(item_vecs), 1))

In [75]:
suser_vecs = scalerUser.transform(user_vec)
sitem_vecs = scalerItem.transform(item_vecs)

In [76]:
suser_vecs[:7]

array([[26.3020864 , -1.2142439 , -7.48604051, -5.88972698,  2.55623364,
        -3.76612966, -3.46393965, -5.68465283, -5.16081591, -1.244141  ,
        -6.75133877,  2.08698324, -2.2559249 , -4.68122296, -4.86517161,
        -5.00476899, -5.67635045]])

In [77]:
len(item_vecs)

847

In [78]:
sitem_vecs.shape

(847, 17)

In [79]:
y_p = recoModel.model.predict([sitem_vecs,suser_vecs])

ValueError: Data cardinality is ambiguous:
  x sizes: 847, 1
Make sure all arrays contain the same number of samples.

In [ ]:
y_p

In [71]:
y_pu = scalerTarget.inverse_transform(y_p)

In [72]:
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #using unscaled vectors for display




In [ ]:
sorted_ypu